In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib as plt
import time
from keras.models import load_model


/Users/ozan/Desktop/Traffic_Sign_Classification_Detection/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Perform histogram equalization on the randomly selected image
def equalize(img):
  img = cv2.equalizeHist(img)
  return img

# Convert the randomly selected image to grayscale
def grayscale(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

# Perform grayscale conversion, histogram equalization and normalization on the whole dataset
def preprocessing(img):
  img = grayscale(img)
  img = equalize(img)
  img = img/255
  return img

In [3]:
best_model = load_model('../CNN_model/cnn_classifier.keras')


In [4]:
# Load weights
path_to_weights = '../yolov4_model/yolov4_tsb_train_best.weights'
# Load configuration file
path_to_cfg = '../yolov4_model/yolov4_tsb_test.cfg'

# Load weights and configuration file into the network
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV) #OPENCV
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16) #OPENCL_FP16

In [5]:
# Get names of all YOLO v4 layers
layers_all = network.getLayerNames()

# Get only detection layers
output_layers = network.getUnconnectedOutLayers()
layers_names_output = [layers_all[i - 1] for i in output_layers]


In [6]:
# Load file with the names of traffic signs

labels = pd.read_csv('../data/signnames.csv')
print(labels)

    ClassId                                           SignName
0         0                               Speed limit (20km/h)
1         1                               Speed limit (30km/h)
2         2                               Speed limit (50km/h)
3         3                               Speed limit (60km/h)
4         4                               Speed limit (70km/h)
5         5                               Speed limit (80km/h)
6         6                        End of speed limit (80km/h)
7         7                              Speed limit (100km/h)
8         8                              Speed limit (120km/h)
9         9                                         No passing
10       10       No passing for vechiles over 3.5 metric tons
11       11              Right-of-way at the next intersection
12       12                                      Priority road
13       13                                              Yield
14       14                                            

In [7]:
# Minimum probability to eliminate weak detections

probability_minimum = 0.6

# Set threshold to filter weak bounding boxes by non-maximum suppression
threshold = 0.4

# Generate colours for bounding boxes
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [10]:

video = cv2.VideoCapture('../data/Filmim.mp4')
# Check if the video capture is opened successfully
if video.isOpened():
    print("Video capture opened successfully.")
    # Get spatial dimensions of the frames
    w = int(video.get(3))
    h = int(video.get(4))
    print(f"Video resolution: {w} x {h}")
else:
    print("Error opening video capture.")
# Writer that will be used to write processed frames
writer = None
# Variables for spatial dimensions of the frames
h, w = None, None

import os

directory_path = '../results/videos'

if os.path.exists(directory_path):
    # Check read permission
    read_permission = os.access(directory_path, os.R_OK)
    print(f"Read Permission: {read_permission}")

    # Check write permission
    write_permission = os.access(directory_path, os.W_OK)
    print(f"Write Permission: {write_permission}")

    # Check execute permission
    execute_permission = os.access(directory_path, os.X_OK)
    print(f"Execute Permission: {execute_permission}")
else:
    print(f"The directory '{directory_path}' does not exist.")


Video capture opened successfully.
Video resolution: 1920 x 1080
Read Permission: True
Write Permission: True
Execute Permission: True


In [11]:
# Set default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variable for counting total processing time
t = 0

# Catch frames in the loop
while True:
    # Capture frames one-by-one
    ret, frame = video.read()

    # If the frame was not retrieved
    if not ret:
        break

    # Get spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slice two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (800, 800), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increase counters
    f += 1
    t += end - start

    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))


    # Lists for detected bounding boxes, confidences, and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Go through all output layers after feed forward pass
    for result in output_from_network:
        # Go through all detections from the current output layer
        for detected_objects in result:
            # Get 80 classes probabilities for the current detected object
            scores = detected_objects[5:]
            # Get the index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Get the value of probability for the defined class
            confidence_current = scores[class_current]

            # Eliminate weak predictions by minimum probability
            if confidence_current > probability_minimum:
                # Scale bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Get top left corner coordinates
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Add results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    # Implement non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Check if there is any detected object been left
    if len(results) > 0:
        # Go through indexes of results
        for i in results.flatten():
            # Bounding box coordinates (width and height)
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Check if the detected sign is a stop sign (assuming class number for stop sign is 14)
            if class_numbers[i] == 14:
                # Write warning text on the frame
                warning_text = "Warning: Stop sign detected!"
                cv2.putText(frame, warning_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Cut fragment with traffic sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
                       
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:

                # Preprocess cut out fragments
                c_ts = np.asarray(c_ts)
                c_ts = cv2.resize(c_ts, (32, 32))
                c_ts = preprocessing(c_ts)
                c_ts = c_ts.reshape(1, 32, 32, 1)

                # Feed to the Clasification model to get predicted label among 43 classes
                scores = best_model.predict(c_ts)

                # Get only class with max confidence
                prediction = np.argmax(scores)

                # Color for current bounding box
                color_box_current = colours[class_numbers[i]].tolist()

                # Draw bounding box on the original current frame
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              color_box_current, 2)

                # Prepare text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction], confidences[i])

                # Put text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_box_current, 2)


    # Initialize writer only once
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('../results/videos/res.mp4', fourcc, 25,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)


# Release video reader and writer
video.release()
writer.release()

Frame number 1 took 1.05929 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame number 2 took 0.67869 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame number 3 took 0.80740 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame number 4 took 0.92322 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame number 5 took 0.66144 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame number 6 took 0.56620 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame number 7 took 0.53225 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame number 8 took 0.59555 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame number 9 took 1.15073 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19